In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
import time
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import random

In [2]:
import os
import shutil

# Ruta relativa del dataset local
dataset_path = './Wonders of World/total'

# Verifica si la ruta existe
if os.path.exists(dataset_path):
    print('Data source import complete.')
else:
    print('Error: La ruta especificada no existe.')


Data source import complete.


In [3]:
# Establecer la semilla
seed = int(time.time())
random.seed(seed)  # Generar una semilla aleatoria basada en el tiempo actual

seed = 1728676400  # Semilla fija obtenida previamente

# Establecer la semilla global para TensorFlow y otras librerías
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

def create_directories(source_dir, dest_dirs):
    """
    Crea directorios en los destinos especificados con los mismos nombres de carpetas que en el directorio de origen.

    :param source_dir: Directorio de origen que contiene las carpetas.
    :param dest_dirs: Lista de directorios de destino donde se crearán las carpetas.
    """
    for category in os.listdir(source_dir):
        category_path = os.path.join(source_dir, category)
        if os.path.isdir(category_path):
            for dest_dir in dest_dirs:
                dest_category_dir = os.path.join(dest_dir, category)
                os.makedirs(dest_category_dir, exist_ok=True)

# Directorio de origen (ruta completa)
source_dir = './Wonders of World/total'

# Directorios de destino (rutas completas)
train_dir = './Wonders of World/train'
val_dir = './Wonders of World/val'
test_dir = './Wonders of World/test'

# Crear directorios
create_directories(source_dir, [train_dir, val_dir, test_dir])

def split_data(source_dir, train_dir, val_dir, test_dir, val_split=0.15, test_split=0.15):
    """
    Divide las imágenes en conjuntos de entrenamiento, validación y prueba, y las mueve a los directorios correspondientes.

    :param source_dir: Directorio de origen que contiene las imágenes.
    :param train_dir: Directorio de destino para el conjunto de entrenamiento.
    :param val_dir: Directorio de destino para el conjunto de validación.
    :param test_dir: Directorio de destino para el conjunto de prueba.
    :param val_split: Proporción de datos para validación.
    :param test_split: Proporción de datos para prueba.
    """
    for category in os.listdir(source_dir):
        category_path = os.path.join(source_dir, category)
        if os.path.isdir(category_path):
            images = os.listdir(category_path)
            random.shuffle(images)
            num_images = len(images)
            num_test = int(num_images * test_split)
            num_val = int(num_images * val_split)
            num_train = num_images - num_test - num_val

            train_images = images[:num_train]
            val_images = images[num_train:num_train + num_val]
            test_images = images[num_train + num_val:]

            # Mover imágenes a sus respectivos directorios
            for image in train_images:
                shutil.move(os.path.join(category_path, image), os.path.join(train_dir, category, image))
            for image in val_images:
                shutil.move(os.path.join(category_path, image), os.path.join(val_dir, category, image))
            for image in test_images:
                shutil.move(os.path.join(category_path, image), os.path.join(test_dir, category, image))

# Dividir los datos
split_data(source_dir, train_dir, val_dir, test_dir)

print("División de datos completada.")


División de datos completada.


Si la celda anterior da como resultado de la ejecución "Failed to load (likely expired)" hay que generar un nuevo notebook desde el sitio de kaggle del dataset y cickear "File" luego "Open in Colab" para generar un nuevo link para descargar los datos desde Colab

In [4]:
# Generadores para entrenar, validar y testear
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_val = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)

train_generator = datagen_train.flow_from_directory(
   train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    seed=seed
)

val_generator = datagen_val.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    seed=seed
)

test_generator = datagen_test.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    seed=seed
)

Found 2706 images belonging to 12 classes.
Found 570 images belonging to 12 classes.
Found 570 images belonging to 12 classes.


In [6]:
model = models.Sequential([
    layers.InputLayer(shape=(150, 150, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(12, activation='softmax'),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,829,900 (18.42 MB)

 Trainable params: 4,829,900 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps= val_generator.samples // val_generator.batch_size,
    epochs=30
)

c:\Users\Lautaro\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 36s 418ms/step - accuracy: 0.1895 - loss: 2.3507 - val_accuracy: 0.4779 - val_loss: 1.5993
Epoch 2/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6875 - loss: 1.2531 - val_accuracy: 0.5000 - val_loss: 1.6219
Epoch 3/30


c:\Users\Lautaro\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


84/84 ━━━━━━━━━━━━━━━━━━━━ 19s 228ms/step - accuracy: 0.5382 - loss: 1.4515 - val_accuracy: 0.5938 - val_loss: 1.2942
Epoch 4/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - accuracy: 0.6250 - loss: 1.0105 - val_accuracy: 0.6538 - val_loss: 1.0789
Epoch 5/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 19s 229ms/step - accuracy: 0.6585 - loss: 1.0785 - val_accuracy: 0.7040 - val_loss: 0.9959
Epoch 6/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - accuracy: 0.7188 - loss: 0.8005 - val_accuracy: 0.7692 - val_loss: 1.1286
Epoch 7/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 19s 228ms/step - accuracy: 0.8027 - loss: 0.6062 - val_accuracy: 0.7353 - val_loss: 0.9344
Epoch 8/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - accuracy: 0.7500 - loss: 0.8275 - val_accuracy: 0.7308 - val_loss: 1.7132
Epoch 9/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 20s 240ms/step - accuracy: 0.9076 - loss: 0.3158 - val_accuracy: 0.7702 - val_loss: 0.9877
Epoch 10/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step - accuracy: 0.9375 - loss: 0.1860 - val_accuracy: 0.6923 -

In [9]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('Test accuracy:', test_acc)

# Guardar el modelo en formato .keras
model.save('./modelLautaro.keras')

# Convertir el modelo a TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Guardar el modelo convertido en formato .tflite
with open('./modelLautaro.tflite', 'wb') as f:
    f.write(tflite_model)


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - accuracy: 0.8266 - loss: 1.3292
Test accuracy: 0.810661792755127
INFO:tensorflow:Assets written to: C:\Users\Lautaro\AppData\Local\Temp\tmp9gs5uhty\assets


INFO:tensorflow:Assets written to: C:\Users\Lautaro\AppData\Local\Temp\tmp9gs5uhty\assets


Saved artifact at 'C:\Users\Lautaro\AppData\Local\Temp\tmp9gs5uhty'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='keras_tensor_10')
Output Type:
  TensorSpec(shape=(None, 12), dtype=tf.float32, name=None)
Captures:
  2782710205520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2782710205904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2782710206096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2782710207056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2782710207248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2782710207824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2782710208016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2782710208592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2782710208784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2782710209360: TensorSpec(shape=(), dtype=tf.resource, name=None)
